<a href="https://colab.research.google.com/github/chueneelvin/Databricks/blob/main/Build_a_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Overview
We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation.

# Install required packages

In [28]:
!pip install langchain langsmith langchain-groq langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00


# Importing the dependecies

In [7]:
from langchain_groq import ChatGroq
import os


# Setting the environmental variables (Langsmith)

In [4]:
from google.colab import userdata
os.environ['LANGCHAIN_TRACING_V2'] = userdata.get('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')

# Setting the environmental variables (LLM)

In [6]:
# Get the API key from user data
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY') # https://console.groq.com/keys

# Initalise the LLM model (Llama 3)

In [20]:
model = ChatGroq(model_name="llama3-70b-8192")

In [25]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 15, 'total_tokens': 41, 'completion_time': 0.083455247, 'prompt_time': 0.000303497, 'queue_time': 0.012779256000000001, 'total_time': 0.083758744}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_c1a4bcec29', 'finish_reason': 'stop', 'logprobs': None}, id='run-2d341567-a758-4ad2-95c3-6eed06e0b49f-0', usage_metadata={'input_tokens': 15, 'output_tokens': 26, 'total_tokens': 41})

The model on its own does not have any concept of state. For example, if you ask a followup question: "What's my name?"

In [26]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I don't know your name. I'm a large language model, I don't have the ability to retain personal information about individuals, and I don't have the ability to see or access any personal data. Each time you interact with me, it's a new conversation and I don't have any prior knowledge about you. If you'd like to share your name with me, I'd be happy to chat with you!", response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 15, 'total_tokens': 106, 'completion_time': 0.291387114, 'prompt_time': 0.000652824, 'queue_time': 0.013588854000000001, 'total_time': 0.292039938}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_c1a4bcec29', 'finish_reason': 'stop', 'logprobs': None}, id='run-9a1b43e5-6602-4e0e-94e6-8c451c015fe1-0', usage_metadata={'input_tokens': 15, 'output_tokens': 91, 'total_tokens': 106})

We can see that it doesn't take the previous conversation turn into context, and cannot answer the question. This makes for a terrible chatbot experience!

To get around this, we need to pass the entire conversation history into the model. Let's see what happens when we do that:



In [27]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob!', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 40, 'total_tokens': 46, 'completion_time': 0.0196244, 'prompt_time': 0.001193089, 'queue_time': 0.01426751, 'total_time': 0.020817489}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_c1a4bcec29', 'finish_reason': 'stop', 'logprobs': None}, id='run-1daa7f1c-69d7-48d0-9f5d-0b9f72efcc6c-0', usage_metadata={'input_tokens': 40, 'output_tokens': 6, 'total_tokens': 46})

And now we can see that we get a good response!

This is the basic idea underpinning a chatbot's ability to interact conversationally. So how do we best implement this?

# Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

First, let's make sure to install langchain-community, as we will be using an integration in there to store message history.

A key part here is the function we pass into as the get_session_history. This function is expected to take in a session_id and return a Message History object. This session_id is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain

In [29]:
from langchain_core.chat_history import (BaseChatMessageHistory,InMemoryChatMessageHistory)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

We now need to create a config that we pass into the runnable every time. This config contains information that is not part of the input directly, but is still useful. In this case, we want to include a session_id. This should look like:

In [32]:
config = {"configurable": {"session_id": "abc2"}}

In [33]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

"Hi Bob! It's nice to meet you! Is there something I can help you with or would you like to chat about something in particular?"

In [34]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! whats my name?")],
    config=config,
)

response.content

'I remember! Your name is Bob!'

In [35]:
response = with_message_history.invoke(
    [HumanMessage(content="How did manage to remember my name?")],
    config=config,
)

response.content

'I\'m a large language model, I don\'t have personal memory like humans do, but I can keep track of the conversation history. When you introduced yourself as "Bob" at the start of our conversation, I stored that information in my short-term memory, which allows me to recall it later in the conversation. It\'s a simple way for me to keep context and personalize our interaction!'

In [36]:
response = with_message_history.invoke(
    [HumanMessage(content="cool i like that")],
    config=config,
)

response.content

"Me too, Bob! It's one of the ways I try to make conversations feel more natural and friendly. I'm always happy to chat with you, and I'll do my best to remember important details like your name throughout our conversation!"

Great! Our chatbot now remembers things about us. If we change the config to reference a different session_id, we can see that it starts the conversation fresh.

In [37]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, but I don't have access to personal information about you, so I don't know your name. I'm a large language model, I don't have the ability to retain information about individual users. Each time you interact with me, it's a new conversation and I don't have any prior knowledge about you. If you want to share your name with me, I'd be happy to chat with you!"

However, we can always go back to the original conversation (since we are persisting it in a database)

In [38]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'I remember! Your name is Bob!'

# Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

First, let's add in a system message. To do this, we will create a ChatPromptTemplate. We will utilize MessagesPlaceholder to pass all the messages in.

In [39]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [40]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

"Hi Bob! It's nice to meet you! Is there something I can help you with or would you like to chat about something in particular? I'm all ears!"

# Streaming
Now we've got a function chatbot. However, one really important UX consideration for chatbot application is streaming. LLMs can sometimes take a while to respond, and so in order to improve the user experience one thing that most application do is stream back each token as it is generated. This allows the user to see progress.

It's actually super easy to do this!

All chains expose a .stream method, and ones that use message history are no different. We can simply use that method to get back a streaming response

In [41]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

KeyError: 'input'